Тема “POS-tagger и NER”

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы
 
Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov
написать свой нер попробовать разные подходы
передаём в сетку токен и его соседей
передаём в сетку только токен
свой вариант
сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [1]:
import nltk

In [25]:
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

# Читаю данные

In [3]:
from corus import load_rudrec
from razdel import tokenize

path = 'rudrec_annotated.json'
records = load_rudrec(path)
next(records)

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

In [4]:
docs = []
for rec in load_rudrec(path):
    words = []
    labels = []
    idx_ent = -1
    len_ents = len(rec.entities)
    rec_entities = sorted(rec.entities, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue

        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.end):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True

        if (token.start >= ent.start) and (token.stop <= ent.end):
                type_ent = 'B-' + ent.entity_type if is_start else 'I-' + ent.entity_type
                is_start = False
        words.append(token.text)
        labels.append(type_ent)
    
    docs.append([words, labels])

# Преобразую в список списков, где токен соответствует тегу

In [19]:
tagged_sents = []
for i in docs:
    list_1 = i[0]
    list_2 = i[1]
#     print(list_1, list_2)
    zip_sents = list(zip(list_1, list_2))
    tagged_sents.append(zip_sents)

In [20]:
tagged_sents[0]

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'B-ADR'),
 ('покрылся', 'I-ADR'),
 (',', 'OUT'),
 ('глаза', 'B-ADR'),
 ('опухли', 'I-ADR'),
 (',', 'OUT'),
 ('сверху', 'OUT'),
 ('и', 'OUT'),
 ('снизу', 'OUT'),
 ('на', 'B-ADR'),
 ('веках', 'I-ADR'),
 ('высыпала', 'I-ADR'),
 ('сыпь', 'I-ADR'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'OUT'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', 'B-Drugform'),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', 'OUT'),
 ('!', 'OUT'),
 ('1', 'OUT'),
 ('Чем', 'OUT'),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

# Подсчитываю количество тегов

In [22]:
count_dict = {}
for list_ in tagged_sents:
    for tag in list_:
        if tag[1] in count_dict:
            count_dict[tag[1]] += 1
        else:
            count_dict[tag[1]] = 1

In [23]:
count_dict

{'OUT': 61585,
 'B-ADR': 719,
 'I-ADR': 746,
 'B-Drugform': 829,
 'B-DI': 1398,
 'I-DI': 875,
 'B-Drugname': 1034,
 'B-Drugclass': 328,
 'I-Drugname': 119,
 'B-Finding': 233,
 'I-Finding': 112,
 'I-Drugform': 6,
 'I-Drugclass': 1}

# Делю на train, test

In [38]:
train_data = tagged_sents[1:int(len(tagged_sents) * 0.9)]
test_data = tagged_sents[int(len(tagged_sents) * 0.9):]
test_sent = tagged_sents[0]
test_data[0]

[('ПЛЮСЫ', 'OUT'), (':', 'OUT'), ('1', 'OUT'), ('.', 'OUT')]

In [52]:
test_sent_out = []
for t in test_sent:
    test_sent_out.append(t[0])

In [53]:
test_sent_out

['нам',
 'прописали',
 ',',
 'так',
 'мой',
 'ребенок',
 'сыпью',
 'покрылся',
 ',',
 'глаза',
 'опухли',
 ',',
 'сверху',
 'и',
 'снизу',
 'на',
 'веках',
 'высыпала',
 'сыпь',
 ',',
 '(',
 '8',
 'месяцев',
 'сыну',
 ')',
 'А',
 'от',
 'виферона',
 'такого',
 'не',
 'было',
 '...',
 'У',
 'кого',
 'ещё',
 'такие',
 'побочки',
 ',',
 'отзовитесь',
 '!',
 '1',
 'Чем',
 'спасались',
 '?']

# DefaultTagger

In [54]:
default_tagger = nltk.DefaultTagger('OUT')
display(default_tagger.tag(test_sent_out), default_tagger.evaluate(test_data))

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'OUT'),
 ('покрылся', 'OUT'),
 (',', 'OUT'),
 ('глаза', 'OUT'),
 ('опухли', 'OUT'),
 (',', 'OUT'),
 ('сверху', 'OUT'),
 ('и', 'OUT'),
 ('снизу', 'OUT'),
 ('на', 'OUT'),
 ('веках', 'OUT'),
 ('высыпала', 'OUT'),
 ('сыпь', 'OUT'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'OUT'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', 'OUT'),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', 'OUT'),
 ('!', 'OUT'),
 ('1', 'OUT'),
 ('Чем', 'OUT'),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

0.9041306168015629

# UnigramTagger

In [55]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent_out), unigram_tagger.evaluate(test_data))

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'I-ADR'),
 ('покрылся', None),
 (',', 'OUT'),
 ('глаза', 'OUT'),
 ('опухли', None),
 (',', 'OUT'),
 ('сверху', None),
 ('и', 'OUT'),
 ('снизу', None),
 ('на', 'OUT'),
 ('веках', None),
 ('высыпала', 'OUT'),
 ('сыпь', 'B-ADR'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'OUT'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', None),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', None),
 ('!', 'OUT'),
 ('1', 'OUT'),
 ('Чем', None),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

0.8444041306168015

# BigramTagger

In [56]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent_out), bigram_tagger.evaluate(test_data))

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'I-ADR'),
 ('покрылся', None),
 (',', 'OUT'),
 ('глаза', 'OUT'),
 ('опухли', None),
 (',', 'OUT'),
 ('сверху', None),
 ('и', 'OUT'),
 ('снизу', None),
 ('на', 'OUT'),
 ('веках', None),
 ('высыпала', 'OUT'),
 ('сыпь', 'B-ADR'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'OUT'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', None),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', None),
 ('!', 'OUT'),
 ('1', 'OUT'),
 ('Чем', None),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

0.8434272955623779

# TrigramTagger

In [57]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent_out), trigram_tagger.evaluate(test_data))

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'I-ADR'),
 ('покрылся', None),
 (',', 'OUT'),
 ('глаза', 'OUT'),
 ('опухли', None),
 (',', 'OUT'),
 ('сверху', None),
 ('и', 'OUT'),
 ('снизу', None),
 ('на', 'OUT'),
 ('веках', None),
 ('высыпала', 'OUT'),
 ('сыпь', 'B-ADR'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'OUT'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', None),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', None),
 ('!', 'OUT'),
 ('1', 'OUT'),
 ('Чем', None),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

0.8452414178063076

In [47]:
count_dict

{'OUT': 61585,
 'B-ADR': 719,
 'I-ADR': 746,
 'B-Drugform': 829,
 'B-DI': 1398,
 'I-DI': 875,
 'B-Drugname': 1034,
 'B-Drugclass': 328,
 'I-Drugname': 119,
 'B-Finding': 233,
 'I-Finding': 112,
 'I-Drugform': 6,
 'I-Drugclass': 1}

In [58]:
patterns = [
    (r'.*ing$', 'I-ADR'),                # gerunds
    (r'.*ed$', 'B-ADR'),                 # simple past
    (r'.*es$', 'B-Drugform'),                 # 3rd singular present
    (r'.*ould$', 'B-DI'),                # modals
    (r'.*\'s$', 'I-DI'),                # possessive nouns
    (r'.*s$', 'B-Drugname'),                  # plural nouns
    (r'^-?[0-9]+(\.[0-9]+)?$', 'Drugclass'),  # cardinal numbers
    (r'.*', 'OUT'),                      # nouns (default) 
    (r'.*ment$', 'OUT'),                # i.e. wonderment 
    (r'.*ful$', 'B-Finding')                  # i.e. wonderful 
]
regexp_tagger = RegexpTagger(patterns)
display(regexp_tagger.tag(test_sent_out), regexp_tagger.evaluate(test_data))

[('нам', 'OUT'),
 ('прописали', 'OUT'),
 (',', 'OUT'),
 ('так', 'OUT'),
 ('мой', 'OUT'),
 ('ребенок', 'OUT'),
 ('сыпью', 'OUT'),
 ('покрылся', 'OUT'),
 (',', 'OUT'),
 ('глаза', 'OUT'),
 ('опухли', 'OUT'),
 (',', 'OUT'),
 ('сверху', 'OUT'),
 ('и', 'OUT'),
 ('снизу', 'OUT'),
 ('на', 'OUT'),
 ('веках', 'OUT'),
 ('высыпала', 'OUT'),
 ('сыпь', 'OUT'),
 (',', 'OUT'),
 ('(', 'OUT'),
 ('8', 'Drugclass'),
 ('месяцев', 'OUT'),
 ('сыну', 'OUT'),
 (')', 'OUT'),
 ('А', 'OUT'),
 ('от', 'OUT'),
 ('виферона', 'OUT'),
 ('такого', 'OUT'),
 ('не', 'OUT'),
 ('было', 'OUT'),
 ('...', 'OUT'),
 ('У', 'OUT'),
 ('кого', 'OUT'),
 ('ещё', 'OUT'),
 ('такие', 'OUT'),
 ('побочки', 'OUT'),
 (',', 'OUT'),
 ('отзовитесь', 'OUT'),
 ('!', 'OUT'),
 ('1', 'Drugclass'),
 ('Чем', 'OUT'),
 ('спасались', 'OUT'),
 ('?', 'OUT')]

0.8900362824448786

# Комбинация тэггеров

In [59]:
from nltk.tag import TrigramTagger 

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('OUT') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

0.935947530002791

# вывод

Комбинация тэггеров дала немного лучший результат, чем UnigramTagger, BigramTagger по отдельности

# Создание тэггера Русских имен

Мы можем самостоятельно написать необходимый тэггер. Допустим, мы хотим тэггировать имена людей в тексте. Для этого берем корпус имен , создаем класс тэггера, задаем логику: если слово входит в множество имен, присваиваем ему тэг имени "Name", иначе тэг "Other".

In [65]:
from russian_names import RussianNames

In [80]:
rus_name = []
rn = RussianNames(count=300, patronymic=False, transliterate=False)
for person in rn:
    rus_name.append(person.split()[0])
#      print(person.split()[0])

In [81]:
rus_name[:5]

['Надежда', 'Иван', 'Карина', 'Иван', 'Жанна']

In [83]:
from nltk.tag import SequentialBackoffTagger
from nltk.corpus import names

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in rus_name])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
             return 'Name'
        else:
             return 'Other'
            
nt = NamesTagger()
print(nt.tag(['Надежда'])) 
print(nt.tag(['Иван'])) 
print(nt.tag(['Карина'])) 
print(nt.tag(['Иван'])) 
print(nt.tag(['Window']))            

[('Надежда', 'Name')]
[('Иван', 'Name')]
[('Карина', 'Name')]
[('Иван', 'Name')]
[('Window', 'Other')]


Задание 2. Проверить насколько хорошо работает NER данные брать из http://www.labinform.ru/pub/named_entities/ проверить NER из nltk/spacy/deeppavlov написать свой нер попробовать разные подходы передаём в сетку токен и его соседей передаём в сетку только токен свой вариант сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

In [85]:
import os

делю данные в папке на текст и токен-тег

In [92]:
files =  os.listdir('Collection5')

In [103]:
files_ann = [i  for i in files if i[-3:] == 'ann']

In [108]:
files_txt = [i  for i in files if i[-3:] == 'txt']

In [110]:
files_txt

['001.txt',
 '002.txt',
 '003.txt',
 '004.txt',
 '005.txt',
 '006.txt',
 '007.txt',
 '008.txt',
 '009.txt',
 '010.txt',
 '011.txt',
 '012.txt',
 '013.txt',
 '014.txt',
 '015 (!).txt',
 '016.txt',
 '017.txt',
 '018.txt',
 '019.txt',
 '020.txt',
 '021.txt',
 '022.txt',
 '023.txt',
 '025.txt',
 '026.txt',
 '027.txt',
 '028.txt',
 '029.txt',
 '030.txt',
 '031.txt',
 '032.txt',
 '033.txt',
 '034.txt',
 '035.txt',
 '036.txt',
 '037.txt',
 '038.txt',
 '039.txt',
 '03_12_12a.txt',
 '03_12_12b.txt',
 '03_12_12c.txt',
 '03_12_12d.txt',
 '03_12_12g.txt',
 '03_12_12h.txt',
 '040.txt',
 '041.txt',
 '042.txt',
 '043.txt',
 '044.txt',
 '045.txt',
 '046.txt',
 '047.txt',
 '048.txt',
 '049.txt',
 '04_02_13a_abdulatipov.txt',
 '04_03_13a_sorokin.txt',
 '04_12_12b.txt',
 '04_12_12d.txt',
 '04_12_12f.txt',
 '04_12_12g.txt',
 '04_12_12h_corr.txt',
 '050.txt',
 '051.txt',
 '052.txt',
 '053.txt',
 '054.txt',
 '055.txt',
 '056.txt',
 '057.txt',
 '058.txt',
 '059.txt',
 '060.txt',
 '061.txt',
 '062.txt',
 '063

In [124]:
with open('001.txt', 'r', encoding = 'utf-8') as f:
        print(f.read())

Россия рассчитывает на конструктивное воздействие США на Грузию

04/08/2008 12:08

МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.

"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.

"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. 


In [231]:
teg_dict = {}
corpus_ann = []
for file in files_ann:
    with open('Collection5/'+file, 'r', encoding = 'utf-8') as f:
        a = f.read()
        a = [i.split('\t') for i in a.split('\n')]
        pattern = r'[^a-zA-Z]'
        for line in a:
            if len(line)> 1:
                token_teg = line[1]
                token_teg = re.sub(pattern, '', token_teg)
                token = line[2]
                teg_dict[token] = token_teg

    # работа с файлом

In [232]:
teg_dict

{'Россия': 'GEOPOLIT',
 'США': 'GEOPOLIT',
 'Грузию': 'GEOPOLIT',
 'МОСКВА': 'LOC',
 'РИА Новости': 'MEDIA',
 'Тбилиси': 'LOC',
 'России': 'GEOPOLIT',
 'Григорий Карасин': 'PER',
 'Дэниэлом Фридом': 'PER',
 'Южной Осетии': 'LOC',
 'Вашингтона': 'GEOPOLIT',
 'МИД': 'ORG',
 'СЕ': 'ORG',
 'Новости-Грузия': 'MEDIA',
 'Совета Европы': 'ORG',
 'Томас Хаммарберг (Thomas Hammarberg)': 'PER',
 'Хаммарберг': 'PER',
 'ООН': 'ORG',
 'ЕС': 'GEOPOLIT',
 'Бишкеке': 'LOC',
 'БИШКЕК': 'LOC',
 'Киргизии': 'GEOPOLIT',
 'МВД': 'ORG',
 'Бишкека': 'LOC',
 'Киргизия': 'GEOPOLIT',
 '"Манас"': 'ORG',
 'Верховного Совета': 'ORG',
 'Аджарской АР': 'LOC',
 'Грузии': 'GEOPOLIT',
 'Михаила Саакашвили': 'PER',
 'Верховного совета': 'ORG',
 'Аджарской автономной республики': 'LOC',
 'Следственное управление при прокуратуре': 'ORG',
 'Якутии': 'LOC',
 'Следственное управление': 'ORG',
 'Следственного комитета при прокуратуре': 'ORG',
 'Российской Федерации': 'GEOPOLIT',
 'Егора Борисова': 'PER',
 'Агентства национальн

In [233]:
corpus

['Россия рассчитывает на конструктивное воздействие США на Грузию\n\n04/08/2008 12:08\n\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\n\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\n\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
 'Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\n\n05/08/2008 10:32\n\nМОСКВА, 5 августа /Новости-Гру

# NLTK

In [153]:
?nltk.pos_tag

In [155]:
nltk.pos_tag(nltk.word_tokenize(corpus[0]), lang='rus')

[('Россия', 'S'),
 ('рассчитывает', 'V'),
 ('на', 'PR'),
 ('конструктивное', 'A=n'),
 ('воздействие', 'S'),
 ('США', 'S'),
 ('на', 'PR'),
 ('Грузию', 'S'),
 ('04/08/2008', 'NUM=ciph'),
 ('12:08', 'NUM=ciph'),
 ('МОСКВА', 'S'),
 (',', 'NONLEX'),
 ('4', 'NUM=ciph'),
 ('авг', 'S'),
 ('-', 'NONLEX'),
 ('РИА', 'S'),
 ('Новости', 'S'),
 ('.', 'NONLEX'),
 ('Россия', 'S'),
 ('рассчитывает', 'V'),
 (',', 'NONLEX'),
 ('что', 'CONJ'),
 ('США', 'S'),
 ('воздействуют', 'V'),
 ('на', 'PR'),
 ('Тбилиси', 'S'),
 ('в', 'PR'),
 ('связи', 'S'),
 ('с', 'PR'),
 ('обострением', 'S'),
 ('ситуации', 'S'),
 ('в', 'PR'),
 ('зоне', 'S'),
 ('грузино-осетинского', 'A=m'),
 ('конфликта', 'S'),
 ('.', 'NONLEX'),
 ('Об', 'PR'),
 ('этом', 'S-PRO'),
 ('статс-секретарь', 'S'),
 ('-', 'NONLEX'),
 ('заместитель', 'S'),
 ('министра', 'S'),
 ('иностранных', 'A=pl'),
 ('дел', 'S'),
 ('России', 'S'),
 ('Григорий', 'S'),
 ('Карасин', 'S'),
 ('заявил', 'V'),
 ('в', 'PR'),
 ('телефонном', 'A=m'),
 ('разговоре', 'S'),
 ('с', 'PR'

In [156]:
nltk.pos_tag(nltk.word_tokenize(corpus[0]), lang='eng')

[('Россия', 'JJ'),
 ('рассчитывает', 'NNP'),
 ('на', 'NNP'),
 ('конструктивное', 'NNP'),
 ('воздействие', 'NNP'),
 ('США', 'NNP'),
 ('на', 'NNP'),
 ('Грузию', 'VBD'),
 ('04/08/2008', 'CD'),
 ('12:08', 'CD'),
 ('МОСКВА', 'NN'),
 (',', ','),
 ('4', 'CD'),
 ('авг', 'SYM'),
 ('-', ':'),
 ('РИА', 'NN'),
 ('Новости', 'NN'),
 ('.', '.'),
 ('Россия', 'JJ'),
 ('рассчитывает', 'NN'),
 (',', ','),
 ('что', 'NNP'),
 ('США', 'NNP'),
 ('воздействуют', 'NNP'),
 ('на', 'NNP'),
 ('Тбилиси', 'NNP'),
 ('в', 'NNP'),
 ('связи', 'NNP'),
 ('с', 'NNP'),
 ('обострением', 'NNP'),
 ('ситуации', 'NNP'),
 ('в', 'NNP'),
 ('зоне', 'NNP'),
 ('грузино-осетинского', 'JJ'),
 ('конфликта', 'NNP'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('статс-секретарь', 'JJ'),
 ('-', ':'),
 ('заместитель', 'NN'),
 ('министра', 'JJ'),
 ('иностранных', 'NNP'),
 ('дел', 'NNP'),
 ('России', 'NNP'),
 ('Григорий', 'NNP'),
 ('Карасин', 'NNP'),
 ('заявил', 'NNP'),
 ('в', 'NNP'),
 ('телефонном', 'NNP'),
 ('разговоре', 'NNP'),
 ('с', 'NNP

In [160]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(corpus[0]))) if hasattr(chunk, 'label') }

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

# Spacy
Spacy значительно быстрее NLTK, так как она написана на Cython и работает с объектами. Для NER Spacy работает как простой классификатор (неглубокая нейронная сеть с одним скрытым слоем). Объект Doc хранит последовательности токенов и все их аннотации.

In [171]:
nlp = spacy.load("ru_core_news_sm")

c:\program files\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ru_core_news_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [172]:
# !pip install -U spacy
#!python -m spacy info
import spacy
from spacy import displacy
# import en_core_web_md
# nlp = spacy.load('ru_core_web_sm')
# nlp = ru_core_web_md.load()
# ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(corpus[0])
displacy.render(article, jupyter=True, style='ent')


# natasha

In [175]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [177]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
doc = Doc(corpus[0])

In [178]:
doc.segment(segmenter)
display(doc.tokens[:5])
display(doc.sents[:5])

[DocToken(stop=6, text='Россия'),
 DocToken(start=7, stop=19, text='рассчитывает'),
 DocToken(start=20, stop=22, text='на'),
 DocToken(start=23, stop=37, text='конструктивное'),
 DocToken(start=38, stop=49, text='воздействие')]

[DocSent(stop=111, text='Россия рассчитывает на конструктивное воздействие..., tokens=[...]),
 DocSent(start=112, stop=233, text='Россия рассчитывает, что США воздействуют на Тбил..., tokens=[...]),
 DocSent(start=234, stop=399, text='Об этом статс-секретарь - заместитель министра ин..., tokens=[...]),
 DocSent(start=401, stop=692, text='"С российской стороны выражена глубокая озабоченн..., tokens=[...]),
 DocSent(start=694, stop=841, text='"Россия уже призвала Тбилиси к ответственной лини..., tokens=[...])]

In [179]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[0]
sent.morph.print()

              Россия PROPN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
        рассчитывает VERB|Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act
                  на ADP
      конструктивное ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Neut|Number=Sing
         воздействие NOUN|Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing
                 США PROPN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur
                  на ADP
              Грузию PROPN|Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
          04/08/2008 NUM
                  12 NUM
                   : PUNCT
                  08 NUM
              МОСКВА PROPN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   , PUNCT
                   4 NUM
                 авг NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
                   - PUNCT
                 РИА PROPN|Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
             Новости PROPN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Plur
       

In [180]:
sent.syntax.print()

        ┌► Россия         nsubj
┌───┌───└─ рассчитывает   
│   │ ┌──► на             case
│   │ │ ┌► конструктивное amod
│ ┌─└►└─└─ воздействие    obl
│ │   └──► США            nmod
│ │     ┌► на             case
│ └──►┌─└─ Грузию         nmod
│     └►┌─ 04/08/2008     appos
│ ┌─┌─┌─└► 12             conj
│ │ │ └──► :              punct
│ │ └────► 08             nmod
│ │        МОСКВА         
│ │     ┌► ,              punct
│ │ ┌─┌►└─ 4              nummod
│ │ └►└─── авг            flat
│ │     ┌► -              punct
│ └──►┌─└─ РИА            parataxis
│     └──► Новости        appos
└────────► .              punct


Модуль извлечения именованных сущностей не зависит от результатов морфологического и синтаксического разбора, его можно использовать отдельно.

In [181]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Россия рассчитывает на конструктивное воздействие США на Грузию
LOC───                                            LOC    LOC───
04/08/2008 12:08
МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют
LOC───          ORG────────  LOC───                   LOC             
 на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского 
    LOC────                                                           
конфликта. Об этом статс-секретарь - заместитель министра иностранных 
дел России Григорий Карасин заявил в телефонном разговоре с 
    LOC─── PER─────────────                                 
заместителем госсекретаря США Дэниэлом Фридом.
                          LOC PER──────────── 
"С российской стороны выражена глубокая озабоченность в связи с новым 
витком напряженности вокруг Южной Осетии, противозаконными действиями 
                            LOC─────────                              
грузинской стороны по наращиванию своих вооруженных сил в регионе, 
бес

Я считаю лучше всего отработала Наташа, она отличила Россию, США от людей и организаций

# NN

In [237]:
import pandas as pd

In [299]:
corpus[0]

'Россия рассчитывает на конструктивное воздействие США на Грузию\n\n04/08/2008 12:08\n\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\n\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\n\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. '

In [284]:
data = pd.DataFrame({'Word': teg_dict.keys(), 'Tag': teg_dict.values()})

In [286]:
data['Tag'].value_counts(normalize=True)

PER         0.557917
ORG         0.253425
LOC         0.113114
GEOPOLIT    0.041398
MEDIA       0.034146
Name: Tag, dtype: float64

создаю 4 словаря , два индекс значение и значение индекс

In [289]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [294]:
# token2idx
print(tag2idx)
idx2tag

{'PER': 0, 'LOC': 1, 'GEOPOLIT': 2, 'ORG': 3, 'MEDIA': 4}


{0: 'PER', 1: 'LOC', 2: 'GEOPOLIT', 3: 'ORG', 4: 'MEDIA'}

для NN нужны числовые данные, преобразую

In [295]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.head()

,Word,Tag,Word_idx,Tag_idx
0,Россия,GEOPOLIT,4330,2
1,США,GEOPOLIT,7087,2
2,Грузию,GEOPOLIT,3240,2
3,МОСКВА,LOC,8550,1
4,РИА Новости,MEDIA,6756,4


In [314]:
data.head()

,Word,Tag,Word_idx,Tag_idx
0,Россия,GEOPOLIT,4330,2
1,США,GEOPOLIT,7087,2
2,Грузию,GEOPOLIT,3240,2
3,МОСКВА,LOC,8550,1
4,РИА Новости,MEDIA,6756,4


In [300]:
corpus_dict = {i : j for i,j in enumerate(corpus)}

делаю df где теги и индексы слов соответствуют тексту

In [309]:
df = pd.DataFrame({'id': corpus_dict.keys(), 'text': corpus_dict.values()})

In [312]:
df.head()

,id,text,Words
0,0,Россия рассчитывает на конструктивное воздейст...,"[Россия, рассчитывает, на, конструктивное, воз..."
1,1,Комиссар СЕ критикует ограничительную политику...,"[Комиссар, СЕ, критикует, ограничительную, пол..."
2,2,"Пулеметы, автоматы и снайперские винтовки изъя...","[Пулеметы,, автоматы, и, снайперские, винтовки..."
3,3,4 октября назначены очередные выборы Верховног...,"[4, октября, назначены, очередные, выборы, Вер..."
4,4,Следственное управление при прокуратуре требуе...,"[Следственное, управление, при, прокуратуре, т..."


In [311]:
df['Words'] = df.text.apply(lambda x: x.split())

In [320]:
df['Tag'] = df.Words.apply(lambda x: [row[1] for index, row in data.iterrows() if row[0] in x])

In [322]:
%time df['Word_idx'] = df.Words.apply(lambda x: [row[2] for index, row in data.iterrows() if row[0] in x])

Wall time: 6min 52s


In [323]:
%time df['Tag_idx'] = df.Words.apply(lambda x: [row[3] for index, row in data.iterrows() if row[0] in x])

Wall time: 5min 10s


In [339]:
df['Sentence #'] = df.id.apply(lambda x: 'Sentence' + str(x+1))

In [340]:
df.columns

Index(['id', 'text', 'Words', 'Tag', 'Word_idx', 'Tag_idx', 'Sentence #'], dtype='object')

In [341]:
df.to_pickle('df.pkl')

In [337]:
# df = pd.read_pickle('df.pkl')

In [342]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [343]:
df.head()

,Sentence #,id,text,Words,Tag,Word_idx,Tag_idx
0,Sentence1,0,Россия рассчитывает на конструктивное воздейст...,"[Россия, рассчитывает, на, конструктивное, воз...","[GEOPOLIT, GEOPOLIT, GEOPOLIT, LOC, GEOPOLIT, ...","[4330, 7087, 3240, 685, 3706, 8264, 9822]","[2, 2, 2, 1, 2, 3, 1]"
1,Sentence2,1,Комиссар СЕ критикует ограничительную политику...,"[Комиссар, СЕ, критикует, ограничительную, пол...","[ORG, PER, ORG, ORG, GEOPOLIT, LOC]","[5439, 4052, 667, 6251, 7110, 1550]","[3, 0, 3, 3, 2, 1]"
2,Sentence3,2,"Пулеметы, автоматы и снайперские винтовки изъя...","[Пулеметы,, автоматы, и, снайперские, винтовки...","[GEOPOLIT, LOC, GEOPOLIT, ORG, GEOPOLIT, ORG]","[7087, 336, 7748, 1698, 4436, 3846]","[2, 1, 2, 3, 2, 3]"
3,Sentence4,3,4 октября назначены очередные выборы Верховног...,"[4, октября, назначены, очередные, выборы, Вер...","[MEDIA, GEOPOLIT, ORG, PER, ORG, GEOPOLIT, LOC]","[226, 8072, 6251, 6427, 5168, 8422, 1550]","[4, 2, 3, 0, 3, 2, 1]"
4,Sentence5,4,Следственное управление при прокуратуре требуе...,"[Следственное, управление, при, прокуратуре, т...","[LOC, GEOPOLIT, GEOPOLIT]","[2905, 1347, 2067]","[1, 2, 2]"


### Разбиваем данные на выборки

In [327]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [329]:
tag2idx

{'PER': 0, 'LOC': 1, 'GEOPOLIT': 2, 'ORG': 3, 'MEDIA': 4}

In [457]:
len(list(set(data['Word'].to_list())))

9928

In [459]:
data['Tag'].value_counts()

PER         5539
ORG         2516
LOC         1123
GEOPOLIT     411
MEDIA        339
Name: Tag, dtype: int64

здесь я делаю длину листа тегов и индексов одинаковой длины, которые соответствуют самой длинной последовательности токенов в одной строке df, и делаю to_categorical, преобразую списки в список из 5 элементов

In [631]:
def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    print('maxlen = {}'.format(maxlen))
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx['PER'])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tags length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(df, data)

maxlen = 41
train_tokens length: 675 
train_tags length: 675 
test_tokens length: 100 
test_tags: 100 
val_tokens: 225 
val_tags: 225


In [632]:
train_tokens

array([[3288, 7667, 6499, ...,    1,    1,    1],
       [3706, 1347,  280, ...,    1,    1,    1],
       [7132, 4753, 2243, ...,    1,    1,    1],
       ...,
       [3706, 1347, 1152, ...,    1,    1,    1],
       [7087,  280, 2054, ...,    1,    1,    1],
       [1347, 8417, 3058, ...,    1,    1,    1]])

In [633]:
train_tokens.shape

(675, 41)

# Строим нейронную сеть

In [634]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [635]:
# фиксируем состояния для воспроизводимости экспериментов
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [636]:
input_dim = len(list(set(data['Word'].to_list()))) + 1
output_dim = 64
input_length = max([len(s) for s in df['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  9929 
output_dim:  64 
input_length:  41 
n_tags:  5


строю структуру NN

In [637]:

def get_bilstm_lstm_model():
    model = Sequential()

    # Слой Embedding
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length, mask_zero= True))

    # Слой bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Слой LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Слой timeDistributed Layer (обеспечивает выход формата many-to-many)
    model.add(TimeDistributed(Dense(n_tags, activation="tanh")))

    #Optimiser 
#     adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [638]:
model_bilstm_lstm = get_bilstm_lstm_model()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_35 (Embedding)    (None, 41, 64)            635456    
                                                                 
 bidirectional_35 (Bidirecti  (None, 41, 128)          66048     
 onal)                                                           
                                                                 
 lstm_71 (LSTM)              (None, 41, 64)            49408     
                                                                 
 time_distributed_35 (TimeDi  (None, 41, 5)            325       
 stributed)                                                      
                                                                 
Total params: 751,237
Trainable params: 751,237
Non-trainable params: 0
_________________________________________________________________


Обучение

In [639]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [642]:
def train_model(X, y, model):
    
    loss = list()
    for i in range(50):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=128, verbose=1, epochs=5, validation_split=0.2)
#         print(hist.history)
        loss.append(hist.history['loss'])
        print(i)
    return loss

In [643]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_37 (Embedding)    (None, 41, 64)            635456    
                                                                 
 bidirectional_37 (Bidirecti  (None, 41, 128)          66048     
 onal)                                                           
                                                                 
 lstm_75 (LSTM)              (None, 41, 64)            49408     
                                                                 
 time_distributed_37 (TimeDi  (None, 41, 5)            325       
 stributed)                                                      
                                                                 
Total params: 751,237
Trainable params: 751,237
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) an

Epoch 3/5
5/5 [==============================] - 1s 186ms/step - loss: 15.0537 - accuracy: 0.0439 - val_loss: 15.2124 - val_accuracy: 0.0379
Epoch 4/5
5/5 [==============================] - 1s 182ms/step - loss: 15.0836 - accuracy: 0.0439 - val_loss: 15.2124 - val_accuracy: 0.0379
Epoch 5/5
5/5 [==============================] - 1s 186ms/step - loss: 15.0719 - accuracy: 0.0439 - val_loss: 15.2154 - val_accuracy: 0.0379
21
Epoch 1/5
5/5 [==============================] - 1s 191ms/step - loss: 15.0639 - accuracy: 0.0436 - val_loss: 15.2183 - val_accuracy: 0.0379
Epoch 2/5
5/5 [==============================] - 1s 176ms/step - loss: 15.0588 - accuracy: 0.0439 - val_loss: 15.2212 - val_accuracy: 0.0379
Epoch 3/5
5/5 [==============================] - 1s 174ms/step - loss: 15.0763 - accuracy: 0.0438 - val_loss: 15.2212 - val_accuracy: 0.0379
Epoch 4/5
5/5 [==============================] - 1s 189ms/step - loss: 15.0640 - accuracy: 0.0438 - val_loss: 15.2212 - val_accuracy: 0.0379
Epoch 5/5


32
Epoch 1/5
5/5 [==============================] - 1s 190ms/step - loss: 15.0581 - accuracy: 0.0439 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 2/5
5/5 [==============================] - 1s 177ms/step - loss: 15.0894 - accuracy: 0.0439 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 3/5
5/5 [==============================] - 1s 182ms/step - loss: 15.1091 - accuracy: 0.0438 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 4/5
5/5 [==============================] - 1s 178ms/step - loss: 15.0829 - accuracy: 0.0437 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 5/5
5/5 [==============================] - 1s 183ms/step - loss: 15.0807 - accuracy: 0.0439 - val_loss: 15.2445 - val_accuracy: 0.0379
33
Epoch 1/5
5/5 [==============================] - 1s 185ms/step - loss: 15.0814 - accuracy: 0.0437 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 2/5
5/5 [==============================] - 1s 186ms/step - loss: 15.0974 - accuracy: 0.0439 - val_loss: 15.2445 - val_accuracy: 0.0379
Epoch 3

Epoch 4/5
5/5 [==============================] - 1s 187ms/step - loss: 15.1040 - accuracy: 0.0438 - val_loss: 15.2532 - val_accuracy: 0.0379
Epoch 5/5
5/5 [==============================] - 1s 184ms/step - loss: 15.0909 - accuracy: 0.0435 - val_loss: 15.2532 - val_accuracy: 0.0379
44
Epoch 1/5
5/5 [==============================] - 1s 185ms/step - loss: 15.1033 - accuracy: 0.0438 - val_loss: 15.2532 - val_accuracy: 0.0379
Epoch 2/5
5/5 [==============================] - 1s 190ms/step - loss: 15.0931 - accuracy: 0.0436 - val_loss: 15.2561 - val_accuracy: 0.0379
Epoch 3/5
5/5 [==============================] - 1s 179ms/step - loss: 15.1142 - accuracy: 0.0439 - val_loss: 15.2561 - val_accuracy: 0.0379
Epoch 4/5
5/5 [==============================] - 1s 184ms/step - loss: 15.1200 - accuracy: 0.0435 - val_loss: 15.2561 - val_accuracy: 0.0379
Epoch 5/5
5/5 [==============================] - 1s 187ms/step - loss: 15.1084 - accuracy: 0.0439 - val_loss: 15.2561 - val_accuracy: 0.0379
45
Epoch 1

Смотрим результат работы сети

In [644]:
results.head()

,with_add_lstm
0,"[2.0000240802764893, 1.5521878004074097, 1.361..."
1,"[8.541025161743164, 6.610731601715088, 2.94466..."
2,"[1.104355812072754, 0.9859973788261414, 1.6434..."
3,"[14.354920387268066, 14.735048294067383, 14.81..."
4,"[14.959193229675293, 14.962780952453613, 14.96..."


In [645]:
predict = model_bilstm_lstm.predict(test_tokens)

4/4 [==============================] - 1s 14ms/step


In [662]:
num = 4
predicted = np.argmax(predict[num], axis=1)

In [663]:
num = 4
actual = np.argmax(test_tags[num], axis=1)

In [664]:
print(predicted)
actual

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4
 4 4 4 4]


array([3, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [669]:
res ={}
count = 0
for i, j in list(zip(predicted, actual)):
    if i == j:
        print(f'эти Теги модель определила правильно {idx2tag[i]}')

эти Теги модель определила правильно ORG
эти Теги модель определила правильно ORG
эти Теги модель определила правильно ORG


вычисляю precision, recall, f1

In [648]:
TP = tf.math.count_nonzero(predicted * actual)
TN = tf.math.count_nonzero((predicted - 1) * (actual - 1))
FP = tf.math.count_nonzero(predicted * (actual - 1))
FN = tf.math.count_nonzero((predicted - 1) * actual)

In [649]:
print(TP, TN, FP, FN)

tf.Tensor(5, shape=(), dtype=int64) tf.Tensor(38, shape=(), dtype=int64) tf.Tensor(38, shape=(), dtype=int64) tf.Tensor(5, shape=(), dtype=int64)


In [650]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)

In [651]:
print(f'precision : {precision}, recall: {recall}, f1: {f1}')

precision : 0.11627906976744186, recall: 0.5, f1: 0.18867924528301888
